Dio koda preuzet je od FER-a. Ostatak je ili preinačen taj dio koda, ili izvorno napisan kod (autor Šimun Šopar).

In [1]:
import time
import torch.optim
from torch.utils.data import DataLoader
from utils import train, evaluate, compute_representations
import numpy as np
from collections import defaultdict

from simple_embedding import SimpleMetricEmbedding, IdentityModel
from metric_dataset import MNISTMetricDataset


EVAL_ON_TEST = True
EVAL_ON_TRAIN = False
remove_class = None

In [2]:
#@author FER
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"= Using device {device}")



mnist_download_root = "./MNIST data"
ds_train = MNISTMetricDataset(mnist_download_root, split='train')
ds_test = MNISTMetricDataset(mnist_download_root, split='test')
ds_traineval = MNISTMetricDataset(mnist_download_root, split='traineval')

num_classes = 10

print(f"> Loaded {len(ds_train)} training images!")
print(f"> Loaded {len(ds_test)} validation images!")

= Using device cpu
> Loaded 60000 training images!
> Loaded 10000 validation images!


In [3]:
#@author FER
train_loader = DataLoader(
    ds_train,
    batch_size=64,
    shuffle=True,
    pin_memory=True,
    num_workers=4,
    drop_last=True
)

test_loader = DataLoader(
    ds_test,
    batch_size=1,
    shuffle=False,
    pin_memory=True,
    num_workers=1
)

traineval_loader = DataLoader(
    ds_traineval,
    batch_size=1,
    shuffle=False,
    pin_memory=True,
    num_workers=1
)

# Metrički model

In [7]:
#@author FER
emb_size = 32
model = SimpleMetricEmbedding(1, emb_size).to(device)
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=1e-3
)

epochs = 3

In [8]:
#@author FER
for epoch in range(epochs):
    print(f"Epoch: {epoch}")
    t0 = time.time_ns()
    train_loss = train(model, optimizer, train_loader, device)
    print(f"Mean Loss in Epoch {epoch}: {train_loss:.3f}")
    if EVAL_ON_TEST or EVAL_ON_TRAIN:
        print("Computing mean representations for evaluation...")
        representations = compute_representations(model, train_loader, num_classes, emb_size, device)
    if EVAL_ON_TRAIN:
        print("Evaluating on training set...")
        acc1 = evaluate(model, representations, traineval_loader, device)
        print(f"Epoch {epoch}: Train Top1 Acc: {round(acc1 * 100, 2)}%")
    if EVAL_ON_TEST:
        print("Evaluating on test set...")
        acc1 = evaluate(model, representations, test_loader, device)
        print(f"Epoch {epoch}: Test Accuracy: {acc1 * 100:.2f}%")
    t1 = time.time_ns()
    print(f"Epoch time (sec): {(t1-t0)/10**9:.1f}")
    
#torch.save(model.state_dict(), 'model_parameters.pt')

Epoch: 0
Iter: 0, Mean Loss: 45.702
Iter: 100, Mean Loss: 50.245
Iter: 200, Mean Loss: 50.369
Iter: 300, Mean Loss: 50.434
Iter: 400, Mean Loss: 50.365
Iter: 500, Mean Loss: 50.338
Iter: 600, Mean Loss: 50.357
Iter: 700, Mean Loss: 50.290
Iter: 800, Mean Loss: 50.271
Iter: 900, Mean Loss: 50.331
Mean Loss in Epoch 0: 50.364
Computing mean representations for evaluation...
Evaluating on test set...
Epoch 0: Test Accuracy: 65.88%
Epoch time (sec): 71.9
Epoch: 1
Iter: 0, Mean Loss: 51.873
Iter: 100, Mean Loss: 50.159
Iter: 200, Mean Loss: 49.995
Iter: 300, Mean Loss: 50.219
Iter: 400, Mean Loss: 50.261
Iter: 500, Mean Loss: 50.318
Iter: 600, Mean Loss: 50.264
Iter: 700, Mean Loss: 50.221
Iter: 800, Mean Loss: 50.225
Iter: 900, Mean Loss: 50.233
Mean Loss in Epoch 1: 50.186
Computing mean representations for evaluation...
Evaluating on test set...
Epoch 1: Test Accuracy: 65.78%
Epoch time (sec): 100.5
Epoch: 2
Iter: 0, Mean Loss: 52.157
Iter: 100, Mean Loss: 50.148
Iter: 200, Mean Loss: 49

# Usporedba u prostoru slike

In [6]:
#@author Simun Sopar
simple_model = IdentityModel()

t0 = time.time_ns()

if EVAL_ON_TEST or EVAL_ON_TRAIN:
    print("Computing representations for evaluation...")
    representations = defaultdict(list)
    for i, data in enumerate(train_loader):
        anchor, id = data[0], data[-1]
        represent = simple_model.get_features(anchor)
        
        for j in range(len(id)):
            representations[id[j].item()].append(represent[j])

    average_repr = torch.zeros(num_classes, represent.shape[1])
    for k, items in representations.items():
        mean_item = torch.zeros(len(items[0]))
        for item in items:
            mean_item += item
        average_repr[k] = mean_item/len(items)
        
print('Representation computation time (sec):', 10**(-9)*(time.time_ns() - t0))   
    
    


Computing representations for evaluation...
Representation computation time (sec): 9521873600000000000


In [7]:
#@author Simun Sopar
t0 = time.time_ns()

if EVAL_ON_TRAIN:
    print("Evaluating on training set...")
    y = []
    y_pred = []
    for i, data in enumerate(traineval_loader):
        anchor, id = data[0], data[-1]
        represent = torch.reshape(simple_model.get_features(anchor), [-1])
        score = torch.zeros(10)
        for j in range(num_classes):
            score[j] = torch.dot(average_repr[j], represent)
        y_pred.append(torch.argmax(score).item())
        y.append(id.item())
    
    acc = 0
    for i in range(len(y)):
        if(y[i] == y_pred[i]):
            acc += 1
    print('Accuracy: ' + str(acc/len(y)*100) + '%')
        
    
    
if EVAL_ON_TEST:
    print("Evaluating on test set...")
    y = []
    y_pred = []
    for i, data in enumerate(test_loader):
        anchor, id = data[0], data[-1]
        represent = torch.reshape(simple_model.get_features(anchor), [-1])
        score = torch.zeros(10)
        for j in range(num_classes):
            score[j] = torch.dot(average_repr[j], represent)
        y_pred.append(torch.argmax(score).item())
        y.append(id.item())
        
    acc = 0
    for i in range(len(y)):
        if(y[i] == y_pred[i]):
            acc += 1
    print('Accuracy: ' + str(acc/len(y)*100) + '%')
    
    
t1 = time.time_ns()
print(f"Evaluation time (sec): {(t1-t0)/10**9:.1f}")

Evaluating on test set...
Accuracy: 63.09%
Evaluation time (sec): 24.3


# Nedostajuća nula

In [4]:
#@author Simun Sopar

#missing 0

ds_train_missing_0 = MNISTMetricDataset(mnist_download_root, split='train', remove_class = 0)

train_loader_missing_0 = DataLoader(
    ds_train_missing_0,
    batch_size=64,
    shuffle=True,
    pin_memory=True,
    num_workers=4,
    drop_last=True
)


emb_size = 32
model = SimpleMetricEmbedding(1, emb_size).to(device)
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=1e-3
)

epochs = 3

In [6]:
#@author FER
for epoch in range(epochs):
    print(f"Epoch: {epoch}")
    t0 = time.time_ns()
    train_loss = train(model, optimizer, train_loader_missing_0, device)
    print(f"Mean Loss in Epoch {epoch}: {train_loss:.3f}")
    if EVAL_ON_TEST or EVAL_ON_TRAIN:
        print("Computing mean representations for evaluation...")
        representations = compute_representations(model, train_loader, num_classes, emb_size, device)
    if EVAL_ON_TRAIN:
        print("Evaluating on training set...")
        acc1 = evaluate(model, representations, traineval_loader, device)
        print(f"Epoch {epoch}: Train Top1 Acc: {round(acc1 * 100, 2)}%")
    if EVAL_ON_TEST:
        print("Evaluating on test set...")
        acc1 = evaluate(model, representations, test_loader, device)
        print(f"Epoch {epoch}: Test Accuracy: {acc1 * 100:.2f}%")
    t1 = time.time_ns()
    print(f"Epoch time (sec): {(t1-t0)/10**9:.1f}")
    
torch.save(model.state_dict(), 'model_parameters_missing_0.pt')

Epoch: 0
Iter: 0, Mean Loss: 53.013
Iter: 100, Mean Loss: 50.141
Iter: 200, Mean Loss: 50.068
Iter: 300, Mean Loss: 50.177
Iter: 400, Mean Loss: 50.186
Iter: 500, Mean Loss: 50.205
Iter: 600, Mean Loss: 50.236
Iter: 700, Mean Loss: 50.363
Iter: 800, Mean Loss: 50.387
Mean Loss in Epoch 0: 50.382
Computing mean representations for evaluation...
Evaluating on test set...
Epoch 0: Test Accuracy: 67.99%
Epoch time (sec): 66.7
Epoch: 1
Iter: 0, Mean Loss: 45.531
Iter: 100, Mean Loss: 50.205
Iter: 200, Mean Loss: 50.308
Iter: 300, Mean Loss: 50.562
Iter: 400, Mean Loss: 50.625
Iter: 500, Mean Loss: 50.598
Iter: 600, Mean Loss: 50.481
Iter: 700, Mean Loss: 50.420
Iter: 800, Mean Loss: 50.442
Mean Loss in Epoch 1: 50.436
Computing mean representations for evaluation...
Evaluating on test set...
Epoch 1: Test Accuracy: 68.01%
Epoch time (sec): 67.1
Epoch: 2
Iter: 0, Mean Loss: 56.121
Iter: 100, Mean Loss: 50.002
Iter: 200, Mean Loss: 50.255
Iter: 300, Mean Loss: 50.184
Iter: 400, Mean Loss: 50.